In [ ]:
import pandas as pd

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("piterfm/paris-2024-olympic-summer-games")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/paris-2024-olympic-summer-games


In [ ]:
print(path)

/root/.cache/kagglehub/datasets/piterfm/paris-2024-olympic-summer-games/versions/27


In [ ]:
df_athletes = pd.read_csv('/kaggle/input/paris-2024-olympic-summer-games/athletes.csv', encoding='latin1', on_bad_lines='skip',  header=0)
df_athletes = df_athletes[['code', 'name', 'gender', 'country_long', 'disciplines', 'birth_date' ]]
df_athletes.head()
#df_athletes.columns

,code,name,gender,country_long,disciplines,birth_date
0,1532872,ALEKSANYAN Artur,Male,Armenia,['Wrestling'],1991-10-21
1,1532873,AMOYAN Malkhas,Male,Armenia,['Wrestling'],1999-01-22
2,1532874,GALSTYAN Slavik,Male,Armenia,['Wrestling'],1996-12-21
3,1532944,HARUTYUNYAN Arsen,Male,Armenia,['Wrestling'],1999-11-22
4,1532945,TEVANYAN Vazgen,Male,Armenia,['Wrestling'],1999-10-27


In [ ]:
df_medallists = pd.read_csv('/kaggle/input/paris-2024-olympic-summer-games/medallists.csv', encoding='latin1', on_bad_lines='skip',  header=0)
# df_medallists.columns
df_medallists = df_medallists[['medal_date','medal_code', 'name', 'gender', 'country_long', 'discipline', 'url_event', 'code_athlete']]
df_medallists.head()


,medal_date,medal_code,name,gender,country_long,discipline,url_event,code_athlete
0,2024-07-27,1.0,EVENEPOEL Remco,Male,Belgium,Cycling Road,/en/paris-2024/results/cycling-road/men-s-indi...,1903136
1,2024-07-27,2.0,GANNA Filippo,Male,Italy,Cycling Road,/en/paris-2024/results/cycling-road/men-s-indi...,1923520
2,2024-07-27,3.0,van AERT Wout,Male,Belgium,Cycling Road,/en/paris-2024/results/cycling-road/men-s-indi...,1903147
3,2024-07-27,1.0,BROWN Grace,Female,Australia,Cycling Road,/en/paris-2024/results/cycling-road/women-s-in...,1940173
4,2024-07-27,2.0,HENDERSON Anna,Female,Great Britain,Cycling Road,/en/paris-2024/results/cycling-road/women-s-in...,1912525


In [ ]:
# Renommer pour correspondance de clé
df_athletes_renamed = df_athletes.rename(columns={"code": "code_athlete"})

# Jointure
df_combined = pd.merge(
    df_medallists,
    df_athletes_renamed,
    on="code_athlete",
    how="left"
)


In [ ]:
df_combined = df_combined.drop(columns=[
    "name_y", "gender_y",
    "country_long_y"
])

In [ ]:

df_combined = df_combined.rename(columns={
    "name_x": "name",
    "gender_x": "gender",
    "country_long_x": "country_long"
})

df_combined.head()

,medal_date,medal_code,name,gender,country_long,discipline,url_event,code_athlete,disciplines,birth_date
0,2024-07-27,1.0,EVENEPOEL Remco,Male,Belgium,Cycling Road,/en/paris-2024/results/cycling-road/men-s-indi...,1903136,['Cycling Road'],2000-01-25
1,2024-07-27,2.0,GANNA Filippo,Male,Italy,Cycling Road,/en/paris-2024/results/cycling-road/men-s-indi...,1923520,"['Cycling Road', 'Cycling Track']",1996-07-25
2,2024-07-27,3.0,van AERT Wout,Male,Belgium,Cycling Road,/en/paris-2024/results/cycling-road/men-s-indi...,1903147,['Cycling Road'],1994-09-15
3,2024-07-27,1.0,BROWN Grace,Female,Australia,Cycling Road,/en/paris-2024/results/cycling-road/women-s-in...,1940173,['Cycling Road'],1992-07-07
4,2024-07-27,2.0,HENDERSON Anna,Female,Great Britain,Cycling Road,/en/paris-2024/results/cycling-road/women-s-in...,1912525,['Cycling Road'],1998-11-14


In [ ]:
df_combined.describe()

,medal_date,medal_code,code_athlete,birth_date,age_at_medal
count,2315,2314.000000,2.315000e+03,2315,2315.000000
mean,2024-08-05 04:03:12.829373696,2.023336,1.893321e+06,1997-08-06 05:40:52.354211712,26.526566
min,2024-07-27 00:00:00,1.000000,1.532872e+06,1965-11-14 00:00:00,14.000000
25%,2024-08-02 00:00:00,1.000000,1.896552e+06,1994-09-18 00:00:00,23.000000
50%,2024-08-06 00:00:00,2.000000,1.924464e+06,1997-12-27 00:00:00,26.000000
75%,2024-08-09 00:00:00,3.000000,1.950498e+06,2000-12-21 00:00:00,29.000000
max,2024-08-11 00:00:00,3.000000,4.980004e+06,2010-05-12 00:00:00,58.000000
std,NaN,0.820390,2.628276e+05,NaN,4.978307


Cela donne envie de faire une infographie plus jeune vs plus vieux

In [ ]:
df_combined.to_csv('df_combined.csv', index=False)

In [ ]:
df_multi_medals = (
    df_combined.groupby(['code_athlete', 'name', 'country_long'])
    .size()
    .reset_index(name='nombre_medailles')
    .query("nombre_medailles > 1")
    .sort_values("nombre_medailles", ascending=False)
)
df_multi_medals.head()

,code_athlete,name,country_long,nombre_medailles
1400,1945125,ZHANG Yufei,People's Republic of China,6
805,1909294,MARCHAND Leon,France,5
1214,1935923,HUSKE Torri,United States of America,5
1226,1935984,SMITH Regan,United States of America,5
1454,1946218,O'CALLAGHAN Mollie,Australia,5


In [ ]:
df_multi_disc = (
    df_combined.groupby(['code_athlete', 'name', 'country_long'])['discipline']
    .nunique()
    .reset_index(name='nb_disciplines')
    .query("nb_disciplines > 1")
    .sort_values("nb_disciplines", ascending=False)
)
df_multi_disc.head()

,code_athlete,name,country_long,nb_disciplines
1018,1923520,GANNA Filippo,Italy,2
1632,1955079,DYGERT Chloe,United States of America,2
1634,1955081,FAULKNER Kristen,United States of America,2


In [ ]:
df_combined['birth_date'] = pd.to_datetime(df_combined['birth_date'], errors='coerce')
df_combined['medal_date'] = pd.to_datetime(df_combined['medal_date'])
df_gender_medals = (
    df_combined.groupby('gender')['medal_code']
    .count()
    .reset_index(name='nb_medailles')
)

df_combined['age_at_medal'] = (df_combined['medal_date'] - df_combined['birth_date']).dt.days // 365

df_youngest = df_combined.sort_values('age_at_medal').dropna(subset=["age_at_medal"]).head(10)
df_oldest = df_combined.sort_values('age_at_medal', ascending=False).dropna(subset=["age_at_medal"]).head(10)
df_youngest.head()
df_oldest.head()

,medal_date,medal_code,name,gender,country_long,discipline,url_event,code_athlete,disciplines,birth_date,age_at_medal
1126,2024-08-02,2.0,KRAUT Laura,Female,United States of America,Equestrian,/en/paris-2024/results/equestrian/jumping-team...,1951840,['Equestrian'],1965-11-14,58
1222,2024-08-03,3.0,HESTER Carl,Male,Great Britain,Equestrian,/en/paris-2024/results/equestrian/dressage-tea...,1970910,['Equestrian'],1967-06-29,57
339,2024-08-04,2.0,WERTH Isabell,Female,Germany,Equestrian,/en/paris-2024/results/equestrian/dressage-ind...,1914909,['Equestrian'],1969-07-21,55
1215,2024-08-03,1.0,WERTH Isabell,Female,Germany,Equestrian,/en/paris-2024/results/equestrian/dressage-tea...,1914909,['Equestrian'],1969-07-21,55
965,2024-07-30,2.0,DIKEC Yusuf,Male,TÃ¼rkiye,Shooting,/en/paris-2024/results/shooting/10m-air-pistol...,1564893,['Shooting'],1973-01-01,51


In [ ]:
df_unique_athletes = df_combined[['code_athlete', 'country_long']].drop_duplicates()
df_country_athletes = (
    df_unique_athletes.groupby('country_long')
    .size()
    .reset_index(name='nb_athletes_medailles')
    .sort_values('nb_athletes_medailles', ascending=False)
)
df_unique_athletes.head()

,code_athlete,country_long
0,1903136,Belgium
1,1923520,Italy
2,1903147,Belgium
3,1940173,Australia
4,1912525,Great Britain
